# Parquet Row Group

首先我们来了解下 Parquet 中的一些重要概念 (https://github.com/apache/parquet-format):

[EN]

- Block (HDFS block): This means a block in HDFS and the meaning is unchanged for describing this file format. The file format is designed to work well on top of HDFS.
- File: A HDFS file that must include the metadata for the file. It does not need to actually contain the data.
- Row group: A logical horizontal partitioning of the data into rows. There is no physical structure that is guaranteed for a row group. A row group consists of a column chunk for each column in the dataset.
- Column chunk: A chunk of the data for a particular column. They live in a particular row group and are guaranteed to be contiguous in the file.
- Page: Column chunks are divided up into pages. A page is conceptually an indivisible unit (in terms of compression and encoding). There can be multiple page types which are interleaved in a column chunk.

[CN]

考虑一个 DataFrame events, 有 event_id: string(32) 和 event_time: datetime64 两列. 一共 1,000,000 (1M) 行.

- Block: HDFS 分布式文件系统中的数据块, 一般是 128 MB 一个. 也有用 64 MB 或是 256 MB 的.
- File: 一个具体的文件. 里面包含了 metadata.
- Row group: 一个文件内部从逻辑上被分为了很多个 Row Group, 每个 Row Group 的起始位置和结束位置的指针是保存在 File metadata 中的. 一个 Row Group 会包含很多 "行" (虽然数据不是按行存储的) 的数据. 比如我们可以说一个 Parquet 文件有 100 个 Row Group, 每个 Row Group 有 10,000 行数据.
- Column chunk: 一个 Row Group 中会有很多 Column. Column chunk 内的数据是连续存储的. 比如从 0 ~ 1,000,000,000 bytes 是 event_id, 1,000,000,001 ~ 1,005,000,000 是 event_time.

Row Group 对用列式存储进行分析非常有帮助. 我们考虑一个例子: 我们需要读取一个大文件, 其中有个 column 是叫 event_time, 我们只需要 event_time between (start, end) 的数据. 有了 Row Group, 你可以通过 File metadata 直接定位到每个 Row Group 的起始点, 并读取 Row Group 的 metadata, 里面就有这个 Row Group 的 event_time 最大最小值. 这样可以避免扫描许多 Row Group, 从而提高查询速度.

Ref:

- https://arrow.apache.org/docs/python/parquet.html#finer-grained-reading-and-writing

In [1]:
%pip install pathlib_mate
%pip install sfm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
from datetime import datetime
from pathlib_mate import Path
from sfm.timer import DateTimeTimer

In [3]:
n_rows = 1_000_000

def gen_data(n_rows):
    event_type_list = ["event_type_1", "event_type_2", "event_type_3"]
    data_table = pa.table({
        "event_id": np.arange(n_rows),
        "event_type": pa.DictionaryArray.from_arrays(
            np.random.randint(low=0, high=len(event_type_list), size=n_rows), 
            event_type_list,
        ),
        "event_time": pd.date_range("2000-01-01", periods=n_rows, freq="1s")
    })
    return data_table

with DateTimeTimer():
    data_table = gen_data(n_rows)

from 2022-03-16 19:35:40.603381 to 2022-03-16 19:35:40.625431 elapsed 0.022050 second.


In [4]:
data_table.take([0, 1, 2]).to_pandas()

,event_id,event_type,event_time
0,0,event_type_3,2000-01-01 00:00:00
1,1,event_type_3,2000-01-01 00:00:01
2,2,event_type_1,2000-01-01 00:00:02


In [5]:
data_table.take([n_rows - 3, n_rows - 2, n_rows - 1]).to_pandas()

,event_id,event_type,event_time
0,999997,event_type_3,2000-01-12 13:46:37
1,999998,event_type_2,2000-01-12 13:46:38
2,999999,event_type_3,2000-01-12 13:46:39


## Optimize Parquet file Using Row Group

用 Row Group 对 查询进行优化的原理是, 如果你有 1GB 的数据, 而你的查询结果可能只涉及到 10MB 的数据, 那么 Row Group 对于每个 Column 的统计信息就可以帮助跳过哪些不包含目标数据的 Row Group. 很有可能你只需要扫描不到 100MB 数据就可以得到结果了. 而 IO 相比基于内存向量的查询要慢几个数量级, 所以最后扫描的速度相比把整个数据集读到内存要快得多的多.

用 Row Group 进行优化, 建议按照这个流程:

1. 决定哪几个 Column 你会进行 Range Query, 通常是 timestamp 列. 选择最重要的 Range Query Column.
2. 在写入磁盘之前, 对 Range Query Column 进行排序.

知识点:

- ``row_group_size`` 的默认值为 64 * 1024 * 1024 = 64M
- ``pyarrow.parquet.ParquetFile.num_row_groups`` 属性能获得 parquet 文件的 row group 总数
- ``pyarrow.parquet.ParquetFile.metadata.row_group(row_group_id: int)`` 方法能获得具体某个 row group 的 metadata
- ``pyarrow.parquet.ParquetFile.metadata.row_group(row_group_id: int).column(column_id: int)`` 方法能获得具体某个 row group 中某个 column 的 metadata
- ``pyarrow.parquet.ParquetFile.metadata.row_group(row_group_id: int).column(column_id: int).statistics.min / max`` 属性能获得具体某个 row group 中某个 column 的统计最大值, 最小值.

Ref:

- https://arrow.apache.org/docs/python/generated/pyarrow.parquet.write_table.html

In [6]:
pq_file = Path("events_with_row_group.parquet")
pq.write_table(data_table, pq_file.abspath, row_group_size=int(n_rows / 1000))

In [7]:
pq_file.size_in_text

'13.15 MB'

In [8]:
start = datetime(2000, 1, 5)
end = datetime(2000, 1, 6)

with DateTimeTimer():
    parquet_file = pq.ParquetFile(pq_file.abspath)

    tables = list()
    for rg_id in range(parquet_file.num_row_groups):
        rg_meta = parquet_file.metadata.row_group(rg_id)
        event_time_col = rg_meta.column(2)
        if not ((event_time_col.statistics.max < start) or (event_time_col.statistics.min > end)):
            t = parquet_file.read_row_group(rg_id)
            t_filtered = pc.filter(
                t,
                pc.and_(
                    pc.greater_equal(t["event_time"], start),
                    pc.less(t["event_time"], end),
                ),
            )
            tables.append(t_filtered)

    table_filtered = pa.concat_tables(tables)

from 2022-03-16 19:35:40.954322 to 2022-03-16 19:35:40.989486 elapsed 0.035164 second.


In [9]:
print(table_filtered.shape)
print("min event_time is:", pc.min(table_filtered["event_time"]))
print("max event_time is:", pc.max(table_filtered["event_time"]))

(86400, 3)
min event_time is: 2000-01-05 00:00:00
max event_time is: 2000-01-05 23:59:59


In [10]:
with DateTimeTimer():
    t = pq.read_table(pq_file.abspath)
    t_filtered = pc.filter(
        t,
        pc.and_(
            pc.greater_equal(t["event_time"], start),
            pc.less(t["event_time"], end),
        ),
    )

from 2022-03-16 19:35:40.998017 to 2022-03-16 19:35:41.177411 elapsed 0.179394 second.


In [11]:
print(table_filtered.shape)
print("min event_time is:", pc.min(table_filtered["event_time"]))
print("max event_time is:", pc.max(table_filtered["event_time"]))

(86400, 3)
min event_time is: 2000-01-05 00:00:00
max event_time is: 2000-01-05 23:59:59
